# Introduzione ai GIS<br/><small>dati vettoriali</small>

## obiettivi della lezione
- i dati vettoriali e il formato ESRI Shapefile
- il geodataframe in geopandas
- le proiezioni spaziali

**dati utilizzati:**

- [ISTAT](https://www.istat.it/it/archivio/222527) Istituto Italiano di Statistica

### requisiti

- conoscenza di R


### status

_"La terra non è piatta!!!"_


---

![](https://user-images.githubusercontent.com/520851/50280460-e35c1880-044c-11e9-9ed7-cc46754e49db.jpg)

# Spatial feature
https://r-spatial.github.io/sf/

In [1]:
library("sf")

Linking to GEOS 3.12.1, GDAL 3.9.1, PROJ 9.4.1; sf_use_s2() is TRUE



## Importiamo dati geospaziali

Simple Features supporta tutti i formati vettoriali supportati [gdal/ogr](https://www.gdal.org/ogr_formats.html) 

In questo tutorial useremo i dati offerti da [ISTAT](https://istat.it)

### dati delle unità amministrative
[https://www.istat.it/it/archivio/222527](https://www.istat.it/it/archivio/222527)

Il file zip con i confini amministrativi del 2024 è disponibile e questo indirizzo
<br/>
[https://www.istat.it/storage/cartografia/confini_amministrativi/generalizzati/2024/Limiti01012024_g.zip](https://www.istat.it/storage/cartografia/confini_amministrativi/generalizzati/2024/Limiti01012024_g.zip)

### download dei dati

In [2]:
# Controlla se la directory esiste
if (!dir.exists("Limiti01012024_g")) {
  # URL del file zip
  zip_file_url <- "https://www.istat.it/storage/cartografia/confini_amministrativi/generalizzati/2024/Limiti01012024_g.zip"
  zip_file_name <- "Limiti01012024_g.zip"
  # Scarica il file zip
  download.file(zip_file_url, zip_file_name, mode = "wb")

  # Decomprime il file zip
  unzip(zip_file_name)
}

In [3]:
# Mostra tutti i file nella directory corrente e nelle sottodirectory
list_files <- function(path = ".") {
  # Elenca tutti i file in modo ricorsivo
  files <- list.files(path = path, recursive = TRUE, full.names = FALSE)
  # Stampa ciascun file
  for (file in files) {
    print(file)
  }
}

In [4]:
# Elencare i file nella directory corrente
list.files()

[1] "01_SNA_datascience_geospatial.ipynb" "apt.txt"                            
 [3] "environment.yml"                     "hello.R"                            
 [5] "install.R"                           "Limiti01012024_g"                   
 [7] "Limiti01012024_g.zip"                "postBuild"                          
 [9] "README"                              "README.md"                          
[11] "requirements.txt"                    "runtime.txt"

In [5]:
list.files(path = "Limiti01012024_g")

[1] "Com01012024_g"    "ProvCM01012024_g" "Reg01012024_g"    "RipGeo01012024_g"

In [6]:
# Cambia la directory di lavoro a 'Limiti01012024_g'
setwd("Limiti01012024_g")

# Verifica la directory corrente
getwd()

[1] "/home/jovyan/Limiti01012024_g"

In [7]:
list.files()

[1] "Com01012024_g"    "ProvCM01012024_g" "Reg01012024_g"    "RipGeo01012024_g"

Limiti01012023_g => la cartella con i confini generalizzati delle varie unità amministrative italiane a 2023
- RipGeo01012024_g<br/>&nbsp;cartella con i confini delle macroregioni d'Italia
- Reg01012024_g<br/>&nbsp;cartella con i confini regionali
- ProvCM01012024_g<br/>&nbsp;confini provinciali
- Com01012024_g<br/>&nbsp;confini comunali

#### Esploriamo le macroregioni

In [8]:
# Cambia la directory di lavoro a 'Limiti01012024_g'
setwd("RipGeo01012024_g")

In [9]:
getwd()

[1] "/home/jovyan/Limiti01012024_g/RipGeo01012024_g"

In [10]:


# Esegui la funzione per la directory corrente
list_files(".")


[1] "RipGeo01012024_g_WGS84.cpg"
[1] "RipGeo01012024_g_WGS84.dbf"
[1] "RipGeo01012024_g_WGS84.prj"
[1] "RipGeo01012024_g_WGS84.shp"
[1] "RipGeo01012024_g_WGS84.shx"


### ESRI Shapefile

questo è uno *ESRI Shapefile* (un formato vecchio ma molto diffuso per distribuire dati geospaziali vettoriali)

Il formato è proprietario ed alcune specifiche di base sono state rese pubbliche.
Un "ESRI Shapefile" è una collezione di diversi file con lo stesso nome ed estensione diversa.

Le specifiche pubbliche sono per queste estensioni

| estensione | significato       | contenuto del file                                                  |
| --------- | ------------- | -------------------------------------------------------------------- |
| .shp      | shape (forma)         | le geometrie (punti, linee, poligoni)                                |
| .dbf      | file database | gli attributi da assocciati alle geometrie                     |
| .shx      | indici degli shape | gli indici per associare le geometrie agli attributi               |
| .prj      | proiezione    | una stringa di testo strutturata che contiene le regole della proiezione usata |

Pe gestire i dati sono _necessai_ almeno 3 file (*.shp, .shx, and .dbf*), ma il file *.prj* è fondamentale per analizzare le informazioni in particolare quando lo si incrocia con altri.

Oltre ai 4 citati è possibile che un ESRI Shapefile abbia ulteriori file.

Maggiori informazioni [qui](https://www.esri.com/library/whitepapers/pdfs/shapefile.pdf

In [11]:
# Leggere il file shapefile
macroregions <- st_read("RipGeo01012024_g_WGS84.shp")

Reading layer `RipGeo01012024_g_WGS84' from data source 
  `/home/jovyan/Limiti01012024_g/RipGeo01012024_g/RipGeo01012024_g_WGS84.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 5 features and 4 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 313279.3 ymin: 3933826 xmax: 1312016 ymax: 5220292
Projected CRS: WGS 84 / UTM zone 32N


In [12]:
# Verifica il tipo di oggetto
class(macroregions)

[1] "sf"         "data.frame"

Si tratta di un data.frame R con la colonna "geometry" su cui applicare funzioni geospaziali

In [15]:
colnames(macroregions)

[1] "COD_RIP"    "DEN_RIP"    "Shape_Leng" "Shape_Area" "geometry"

In [17]:
print(macroregions)

Simple feature collection with 5 features and 4 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 313279.3 ymin: 3933826 xmax: 1312016 ymax: 5220292
Projected CRS: WGS 84 / UTM zone 32N
  COD_RIP    DEN_RIP Shape_Leng  Shape_Area                       geometry
1       1 Nord-ovest    2330675 57930826800 MULTIPOLYGON (((568227.8 48...
2       2   Nord-est    2313109 62396110675 MULTIPOLYGON (((618343.9 48...
3       3     Centro    2013166 58030006351 MULTIPOLYGON (((876021.9 45...
4       4        Sud    2535023 73781812999 MULTIPOLYGON (((1083359 441...
5       5      Isole    2793918 49930616360 MULTIPOLYGON (((822901.4 39...


In [24]:
# Verifica se ci sono geometrie non valide
st_is_valid(macroregions)

# Identifica geometrie non valide
invalid_geometries <- macroregions[!st_is_valid(macroregions), ]
print(invalid_geometries)

[1]  TRUE FALSE  TRUE FALSE FALSE

Simple feature collection with 3 features and 4 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 426598 ymin: 3933826 xmax: 1312016 ymax: 5220292
Projected CRS: WGS 84 / UTM zone 32N
  COD_RIP  DEN_RIP Shape_Leng  Shape_Area                       geometry
2       2 Nord-est    2313109 62396110675 MULTIPOLYGON (((618343.9 48...
4       4      Sud    2535023 73781812999 MULTIPOLYGON (((1083359 441...
5       5    Isole    2793918 49930616360 MULTIPOLYGON (((822901.4 39...


ERROR: Error in parse(text = x, srcfile = src): <text>:1:34: unexpected ')'
1: st_length(macroregions$geometry[4)
                                     ^


In [27]:
# Calcola la lunghezza delle geometrie valide
geometry_lengths <- st_length(macroregions)
print(geometry_lengths)

Units: [m]
[1] 0 0 0 0 0
